In [9]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity


class Order:  # The Context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
   
    
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0
   
    
def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount
  
    
def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [10]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]

In [11]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [12]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [13]:
# Choosing the best strategy: simple approach
promos = [fidelity_promo, bulk_item_promo, large_order_promo]


def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

# Problem is that when we define new promo we have to remember to
# add it to the list of available promos

In [14]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [ ]:
# Find strategies in module
"""Modules in python are also first class objects
globals()
    returns a dict representing the current global symbol table."""

In [15]:
# We will build promos by introspecting of the module global namespace

promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']


def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

# bit hackish 
# another way is to create a module and put all the strategy 
# functions in there except best promo

In [16]:
promos

[<function __main__.fidelity_promo>,
 <function __main__.bulk_item_promo>,
 <function __main__.large_order_promo>]

In [ ]:
# If we put it in a separate module we can write the following code
# we need to import inspect and promotions modules

promos = [func for name, func in 
          inspect.getmembers(promotions, inspect.isfunciton)]

In [ ]:
# yet more explicit alternative for dynamically collecting 
# promotional discount functions would be to use decorator